In [1]:
import logging

from dask.distributed import Client
from dask import delayed, compute

client = Client(threads_per_worker=2, n_workers=4,silence_logs=logging.ERROR)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 7.65 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43721,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 7.65 GiB
Comm: tcp://127.0.0.1:39175,Total threads: 2
Dashboard: http://127.0.0.1:42175/status,Memory: 1.91 GiB
Nanny: tcp://127.0.0.1:32913,


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

df = pd.read_csv('../dataset/processed_dataset/final_data_S1.csv')
X = df.drop(['class'],axis=1)
y = df['class']

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
def check_param(param_list):
    if param_list['solver'] in ['newton-cg', 'lbfgs', 'sag']:
        return param_list['penalty'] in ['l2', 'none']
    elif param_list['solver'] in ['liblinear']:
        return if param_list['penalty'] in ['l1', 'l2']
    elif param_list['solver'] in ['saga']:
        return if param_list['penalty'] in ['l1', 'l2', 'elasticnet','none']

In [ ]:
param_list ={
    'penalty': ['l1', 'l2', 'none'],
    'C': [0.1, 1, 10, 100, 1000],
    'tol': [1e-4, 1e-3, 1e-2, 1e-1, 1],
    'solver': ['saga', 'liblinear'],
}

from sklearn.model_selection import ParameterGrid
param_dict = ParameterGrid(param_list)

print(len(list(param_dict)))

def RandomCV(param_list,n=100):
    import random
    return (item for item in random.sample(list(param_list),n))

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Testing every combination of parameters with train_6 and test_6
@delayed
def run_model(param):
    if check_param(param):
        model_obj = LogisticRegression(**param, max_iter=1000)
        model_obj.fit(x_train, y_train)
        param.update({
            "score":accuracy_score(y_test, model_obj.predict(x_test))
                      })
        return pd.DataFrame(param,index=[0]).iloc[0:1,:]
    else:
        print(param, "is not valid")

# create dask parallel function to run xgb
dfs = (run_model(each) for each in param_dict)
value = compute(dfs,num_workers=4)
#
final = pd.concat(value[0])
value = []
final.reset_index(drop=True,inplace=True)
# not_used = ['n_jobs']
# final.drop(not_used,axis=1,inplace=True)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Testing every combination of parameters with train_6 and test_6
@delayed
def run_model(param):
    if check_param(param):
        model_obj = LogisticRegression(**param, max_iter=1000)
        model_obj.fit(x_train, y_train)
        param.update({
            "score":accuracy_score(y_test, model_obj.predict(x_test))
                      })
        return pd.DataFrame(param,index=[0]).iloc[0:1,:]
    else:
        print(param, "is not valid")

# create dask parallel function to run xgb
dfs = (run_model(each) for each in param_dict)
value = compute(dfs,num_workers=4)
#
final = pd.concat(value[0])
value = []
final.reset_index(drop=True,inplace=True)
# not_used = ['n_jobs']
# final.drop(not_used,axis=1,inplace=True)

In [10]:
client.close()

In [11]:
final

,C,penalty,solver,tol,score
0,0.1,l1,saga,0.0001,0.750000
1,0.1,l1,saga,0.0010,0.750000
2,0.1,l1,saga,0.0100,0.750000
3,0.1,l1,saga,0.1000,0.750000
4,0.1,l1,saga,1.0000,0.782895
...,...,...,...,...,...
120,1000.0,none,saga,0.0001,0.763158
121,1000.0,none,saga,0.0010,0.763158
122,1000.0,none,saga,0.0100,0.763158
123,1000.0,none,saga,0.1000,0.802632


In [1]:
!pip install dask_saturn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 134.9 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 352.7 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.2/160.2 kB 461.4 kB/s eta 0:00:00a 0:00:01
  Using cached idna-3.3-py3-none-any.whl (61 kB)


In [2]:
from dask_saturn import SaturnCluster
from dask.distributed import Client
from dask import delayed, compute

import os
os.environ['SATURN_BASE_URL'] = "https://app.community.saturnenterprise.io"
os.environ['SATURN_TOKEN'] = "server-4bb72059c7f749af8e03e697f9926845"

cluster = SaturnCluster()
client = Client(cluster)
client

RuntimeError: Missing required environment variable SATURN_BASE_URL.